# About this kernel

- https://www.kaggle.com/xhlulu/disaster-nlp-keras-bert-using-tfhub

I've seen a lot of people pooling the output of BERT, then add some Dense layers. I also saw various learning rates for fine-tuning. In this kernel, I wanted to try some ideas that were used in the original paper that did not appear in many public kernel. Here are some examples:
* *No pooling, directly use the CLS embedding*. The original paper uses the output embedding for the `[CLS]` token when it is finetuning for classification tasks, such as sentiment analysis. Since the `[CLS]` token is the first token in our sequence, we simply take the first slice of the 2nd dimension from our tensor of shape `(batch_size, max_len, hidden_dim)`, which result in a tensor of shape `(batch_size, hidden_dim)`.
* *No Dense layer*. Simply add a sigmoid output directly to the last layer of BERT, rather than experimenting with different intermediate layers.
* *Fixed learning rate, batch size, epochs, optimizer*. As specified by the paper, the optimizer used is Adam, with a learning rate between 2e-5 and 5e-5. Furthermore, they train the model for 3 epochs with a batch size of 32. I wanted to see how well it would perform with those default values.

I also wanted to share this kernel as a **concise, reusable, and functional example of how to build a workflow around the TF2 version of BERT**. Indeed, it takes less than **50 lines of code to write a string-to-tokens preprocessing function and model builder**.

## References

* Source for `bert_encode` function: https://www.kaggle.com/user123454321/bert-starter-inference
* All pre-trained BERT models from Tensorflow Hub: https://tfhub.dev/s?q=bert

In [ ]:
# We will use the official tokenization script created by the Google team
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import time

import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras import callbacks
import tensorflow_hub as hub
from keras.utils import to_categorical

import tokenization

sns.set_style("whitegrid")
notebookstart = time.time()
pd.options.display.max_colwidth = 500

print("Tensorflow Version: ", tf.__version__)
print("TF-Hub version: ", hub.__version__)
print("Eager mode enabled: ", tf.executing_eagerly())
print("GPU available: ", tf.test.is_gpu_available())

# Helper Functions

In [ ]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [ ]:
def build_model(bert_layer, out_channels, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    out = Dense(out_channels, activation='softmax')(clf_output)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=1e-5), loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

# Load and Preprocess

- Load BERT from the Tensorflow Hub
- Load CSV files containing training data
- Load tokenizer from the bert layer
- Encode the text into tokens, masks, and segment flags

In [ ]:
%%time
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)

In [ ]:
MAX_LEN = 64*3
BATCH_SIZE = 8
EPOCHS = 15
SEED = 42
NROWS = None
TEXTCOL = "text"
TARGETCOL = "author"
NCLASS = 3

In [ ]:
train = pd.read_csv("../input/spooky-author-identification/train.zip")
test = pd.read_csv("../input/spooky-author-identification/test.zip")
testdex = test.id
submission = pd.read_csv("../input/spooky-author-identification/sample_submission.zip")

sub_cols = submission.columns

print("Train Shape: {} Rows, {} Columns".format(*train.shape))
print("Test Shape: {} Rows, {} Columns".format(*test.shape))

length_info = [len(x) for x in np.concatenate([train[TEXTCOL].values, test[TEXTCOL].values])]
print("Train Sequence Length - Mean {:.1f} +/- {:.1f}, Max {:.1f}, Min {:.1f}".format(
    np.mean(length_info), np.std(length_info), np.max(length_info), np.min(length_info)))

In [ ]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [ ]:
train_input = bert_encode(train[TEXTCOL].values, tokenizer, max_len=MAX_LEN)
test_input = bert_encode(test[TEXTCOL].values, tokenizer, max_len=MAX_LEN)


label_mapper = {name: i for i,name in enumerate(set(train[TARGETCOL].values))}
num_label = np.vectorize(label_mapper.get)(train[TARGETCOL].values)
train_labels = to_categorical(num_label)

# Model: Build, Train, Predict, Submit

In [ ]:
model = build_model(bert_layer, NCLASS, max_len=MAX_LEN)
model.summary()

In [ ]:
checkpoint = callbacks.ModelCheckpoint('model.h5', monitor='val_loss', save_best_only=True)
es = callbacks.EarlyStopping(monitor='val_loss', min_delta=0.0001,
                             patience=4, verbose=1, mode='min', baseline=None,
                             restore_best_weights=False)


train_history = model.fit(
    train_input, train_labels,
    validation_split=0.2,
    epochs=EPOCHS,
    callbacks=[checkpoint, es],
    batch_size=BATCH_SIZE
)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plot_metrics = ['loss']

f, ax = plt.subplots(1,figsize = [7,4])
for p_i,metric in enumerate(plot_metrics):
    ax.plot(train_history.history[metric], label='Train ' + metric)
    ax.plot(train_history.history['val_' + metric], label='Val ' + metric)
    ax.set_title("Loss Curve - {}".format(metric))
    ax.legend()
plt.show()

In [ ]:
model.load_weights('model.h5')
test_pred = model.predict(test_input)
test_pred.shape

In [ ]:
submission = pd.DataFrame(test_pred, columns=label_mapper.keys())
submission['id'] = testdex

submission = submission[sub_cols]
submission.to_csv('submission_bert.csv', index=False)
print(submission.shape)

In [ ]:
print("Notebook Runtime: %0.2f Minutes"%((time.time() - notebookstart)/60))